In [18]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

基于前两个词，生成第三个词

In [19]:
sentences = [ "i like dog", "i love coffee", "i hate milk"]

word_list = " ".join(sentences).split()
print(word_list)
word_list = list(set(word_list))
print(word_list)
word_dict = {w:i for i,w in enumerate(word_list)}
print(word_dict)
number_dict = {i:w for i,w in enumerate(word_list)}
print(number_dict)
n_class = len(word_dict)
print(n_class)

['i', 'like', 'dog', 'i', 'love', 'coffee', 'i', 'hate', 'milk']
['coffee', 'milk', 'i', 'dog', 'hate', 'love', 'like']
{'coffee': 0, 'milk': 1, 'i': 2, 'dog': 3, 'hate': 4, 'love': 5, 'like': 6}
{0: 'coffee', 1: 'milk', 2: 'i', 3: 'dog', 4: 'hate', 5: 'love', 6: 'like'}
7


In [20]:
n_step = 2  # 考虑前两个词
n_hidden = 2
m = 2

def make_batch(sentences):
    input_batch = []
    target_batch = []
    
    for sen in sentences:
        word = sen.split()
        input = [word_dict[n] for n in word[:-1]]
        target = word_dict[word[-1]]
        
        input_batch.append(input)
        target_batch.append(target)
        
    return input_batch, target_batch

In [21]:
input_batch, target_batch = make_batch(sentences)

print('input_batch:', input_batch)
print('target_batch:', target_batch)

input_batch = Variable(torch.LongTensor(input_batch))
target_batch = Variable(torch.LongTensor(target_batch))

print( input_batch.shape)
print( target_batch.shape)


input_batch: [[2, 6], [2, 5], [2, 4]]
target_batch: [3, 0, 1]
torch.Size([3, 2])
torch.Size([3])


In [22]:
input_size = n_step * m # 2 * 2
hidden_size = n_hidden  # 2 隐藏层单元数为 2

class NNLM(nn.Module):
    def __init__(self, input_size, hidden_size, n_class):
        super(NNLM, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.n_class = n_class
        self.emb = nn.Embedding(n_class, m)  # 7 , 2 词库有7个词，每个词2维
        # num_embeddings: 嵌入层字典的大小（单词本里单词个数） embedding_dim: 每个产出向量的大小
        # pytorch的nn.Embedding层讲 one-hot矩阵 和 C 封装起来了
        
        self.fc1 = nn.Linear(self.input_size, self.hidden_size) # 4 , 2
        self.fc2 = nn.Linear(self.hidden_size, self.n_class) # 2 , 7
    
    def forward(self, x):
        x = self.emb(x) #     # (3,2,7) * (7,2) ==> (3,2,2)
#         print('第一层维度:', x.size())    
        # (3,2,2) 第一个元素是 batchsize的维度，这里的训练集数目是 3；
        # 第二个元素是序列长度，即前n-1个词，这里是2；
        # 第三个元素是每个词的向量维度，这里是2.
        
        x = x.view(-1, self.input_size)  # (3,2,2) ==> (3,4)
#         print('第二层维度:', x.size())   # (3,4) 
        
        x = self.fc1(x)             # (3,4) * (4,2) ==> (3,4)
#         print('第三层维度:', x.size())  # (3,2) 
        
        x = torch.tanh(x)            # (3,2) ==> (3,2) 非线性激活
#         print('第四层维度:', x.size())  # (3,2)
        
        output = self.fc2(x)       # (3,2) * (2,7) ==> (3,7) 全连接层
#         print('第五层维度:', output.size())  # (3,7) 
        return output

model = NNLM(input_size, hidden_size, n_class)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

print(model)

NNLM(
  (emb): Embedding(7, 2)
  (fc1): Linear(in_features=4, out_features=2, bias=True)
  (fc2): Linear(in_features=2, out_features=7, bias=True)
)


In [23]:
for epoch in range(1000):
    output = model(input_batch)
    loss = criterion(output, target_batch)
    
    if (epoch + 1) % 1000 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    


Epoch: 1000 cost = 0.674910


In [25]:
predict = model(input_batch).data.max(1, keepdim=True)[1]
print(predict)

print([sen.split()[:2] for sen in sentences], '->', [number_dict[n.item()] for n in predict.squeeze()]) 


tensor([[3],
        [3],
        [1]])
[['i', 'like'], ['i', 'love'], ['i', 'hate']] -> ['dog', 'dog', 'milk']


In [26]:
print('输入：',input_batch)
print('embedding之后的结果：',model.emb(input_batch))

输入： tensor([[2, 6],
        [2, 5],
        [2, 4]])
embedding之后的结果： tensor([[[-1.6612,  2.5111],
         [ 1.0630, -2.5493]],

        [[-1.6612,  2.5111],
         [ 1.1125, -1.8106]],

        [[-1.6612,  2.5111],
         [-1.6486,  0.6113]]], grad_fn=<EmbeddingBackward>)


In [27]:
print('embedding之后的结果：',model.emb.parameters())
for embed in model.emb.parameters():
    print(embed)  
# nn.Embedding网络中的权重即 本例中词表中所有词汇的词向量

embedding之后的结果： <generator object Module.parameters at 0x0000017C50D849C8>
Parameter containing:
tensor([[-0.8708,  0.8599],
        [-1.9604, -1.1207],
        [-1.6612,  2.5111],
        [ 0.0221,  1.3368],
        [-1.6486,  0.6113],
        [ 1.1125, -1.8106],
        [ 1.0630, -2.5493]], requires_grad=True)
